In [1]:
from pyspark.sql import SparkSession
import pandas as pd
import os
os.environ["PYSPARK_PYTHON"]="/usr/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"]="/usr/bin/python3"
os.environ['PYSPARK_SUBMIT_ARGS'] = '--driver-class-path  /home/sahil/Documents/code/internship/benchmarking/sqljdbc_7.2/enu/mssql-jdbc-7.2.1.jre8.jar --packages org.postgresql:postgresql:42.2.1,mysql:mysql-connector-java:8.0.14,com.microsoft.sqlserver:mssql-jdbc:7.2.1.jre11 --driver-memory 4G pyspark-shell'

spark = SparkSession.builder\
                    .appName('AWS external sources spark')\
                    .master("local[4]")\
                    .getOrCreate()

In [2]:
# spark.stop()

In [3]:
spark

### Load CSV file and cache it

In [4]:
df = spark.read.csv("./cfs.csv", header=True, inferSchema=True)

In [5]:
df.limit(5).toPandas()

,SHIPMT_ID,ORIG_STATE,ORIG_MA,ORIG_CFS_AREA,DEST_STATE,DEST_MA,DEST_CFS_AREA,NAICS,QUARTER,SCTG,MODE,SHIPMT_VALUE,SHIPMT_WGHT,SHIPMT_DIST_GC,SHIPMT_DIST_ROUTED,TEMP_CNTL_YN,EXPORT_YN,EXPORT_CNTRY,HAZMAT,WGT_FACTOR
0,1,25,148,25-148,25,148,25-148,333,2,35,14,2178,11,14,17,N,N,N,N,208.5
1,2,42,428,42-428,6,41740,06-41740,311,3,35,14,344,11,2344,2734,N,N,N,N,193.3
2,3,26,220,26-220,47,314,47-314,322,2,27,4,4197,5134,470,579,N,N,N,N,51.2
3,4,20,556,20-556,20,556,20-556,323,1,29,4,116,6,3,3,N,N,N,N,238.7
4,5,12,99999,12-99999,12,99999,12-99999,4235,3,33,5,388,527,124,201,N,N,N,N,398.1


In [6]:
# repartiton and cache
df = df.repartition(8)
df.cache()
df.count()

4547661

In [ ]:
# df.unpersist()

### Local Postgres Database

For all 4 million rows, it takes about 1 min 45 seconds to run on local machine

In [7]:
# %%timeit -r 3
df.limit(100000).write\
      .format("jdbc")\
      .option("driver", "org.postgresql.Driver")\
      .option("url", "jdbc:postgresql://localhost:5432/postgres")\
      .option("dbtable", "cfs_spark")\
      .option("user", "sahil")\
      .option("password","zxcvbnm")\
      .mode("overwrite")\
      .save()

### AWS Postgres Database

Bear in mid this time is affected by your internet speed and RDS instance type.

In [ ]:
# %%timeit -r 3
df.limit(100000).write\
      .format("jdbc")\
      .option("driver", "org.postgresql.Driver")\
      .option("url", "jdbc:postgresql://postgresdb2.cznthudneeub.eu-west-1.rds.amazonaws.com:5432/postgres")\
      .option("dbtable", "cfs_spark")\
      .option("user", "sahil")\
      .option("password","Asdfg1234!")\
      .mode("overwrite")\
      .save()


In [ ]:
spark.stop()

### Local MySQL Database

In [8]:
# %%timeit -r 3
df.limit(100000).write\
      .format("jdbc")\
      .option("driver", "com.mysql.jdbc.Driver")\
      .option("url", "jdbc:mysql://localhost:3306/mysql")\
      .option("dbtable", "cfs_spark")\
      .option("user", "root")\
      .option("password","zxcvbnm")\
      .mode("overwrite")\
      .save()

### AWS MySQL Database

In [ ]:
%%timeit -r 3
df.limit(100000).write\
      .format("jdbc")\
      .option("driver", "com.mysql.jdbc.Driver")\
      .option("url", "jdbc:mysql://mysql2.cznthudneeub.eu-west-1.rds.amazonaws.com:3306/sahil")\
      .option("dbtable", "cfs_spark")\
      .option("user", "sahil")\
      .option("password","Asdfg1234!")\
      .mode("overwrite")\
      .save()

### SQL Server Local

Download SQLServer Jar from [here](https://docs.microsoft.com/en-us/sql/connect/jdbc/download-microsoft-jdbc-driver-for-sql-server?view=sql-server-2017)

In [ ]:
##
## Dont Run
## 
##### include the jar from the above downloaded packages as --jars option
os.environ['PYSPARK_SUBMIT_ARGS'] = = '--driver-class-path /home/sahil/Desktop/sqljdbc_7.2/enu/mssql-jdbc-7.2.1.jre8.jar'

In [10]:
# %%timeit -r 3
df.limit(100000).write.format("jdbc") \
    .option("url", "jdbc:sqlserver://localhost:1433;databaseName=sparkdemodb") \
    .option("dbtable", "cfs") \
    .option("user", "SA") \
    .option("password", "Asdfg1234!")\
    .mode("overwrite")\
    .save()